In [1]:
import SimpleITK as sitk
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np
import glob
import matplotlib
from utils import (
    load_itk, resample, 
    gen_coordinate_and_mask,
)
from skimage import measure
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd

### HC dataset

In [2]:
# MASK_PATH = r'/data/HC1/*/103*'
# mask_list = glob.glob(MASK_PATH)
# print(len(mask_list))
# print(mask_list[0])

### MSD dataset

In [3]:
MASK_PATH = r'/data/MSD/Task06_Lung/labelsTr/lung*'
mask_list = glob.glob(MASK_PATH)
print(len(mask_list))
print(mask_list[0])

63
/data/MSD/Task06_Lung/labelsTr/lung_001.nii.gz


### batch execution

In [4]:
nodule_coordinates = []
dataset = 'MSD'
for mask in tqdm(mask_list):
    
    img, origin, spacing = load_itk(mask)
    nodule_mask, spacing2 = resample(img, spacing)

    coord_nodule_mask = gen_coordinate_and_mask(nodule_mask)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    p = nodule_mask.transpose(2, 1, 0)
    verts, faces, normals, values = measure.marching_cubes(p, 0)
    mesh = Poly3DCollection(verts[faces], alpha=0.7)
    face_color = [1, 0, 0, 0.7] # red
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    p = coord_nodule_mask['nodule_mask'].transpose(2, 1, 0)
    verts, faces, normals, values = measure.marching_cubes(p, 0)
    mesh = Poly3DCollection(verts[faces], alpha=0.20)
    face_color = [0.45, 0.45, 0.75] # blue
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)
    
    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    raw_name = None
    if dataset == 'HC':
        raw_name = mask.split('/')[3]
    if dataset == 'MSD':
        raw_name = mask.split('/')[5].split('.')[0]
    plt.savefig(
        '/data/' + dataset + '-nodules-coordinates-preview/' + raw_name + '.png'
    )
    plt.close()
    matplotlib.use('Agg')

    nodule_coordinates.append({
        "x_min": coord_nodule_mask["x_min"],
        "y_min": coord_nodule_mask["y_min"],
        "z_min": coord_nodule_mask["z_min"],
        "x_max": coord_nodule_mask["x_max"],
        "y_max": coord_nodule_mask["y_max"],
        "z_max": coord_nodule_mask["z_max"]
    })

    plt.show()

nodule_coordinates

100%|██████████| 63/63 [1:51:00<00:00, 105.73s/it]  


[{'x_min': 231,
  'y_min': 178,
  'z_min': 78,
  'x_max': 248,
  'y_max': 192,
  'z_max': 95},
 {'x_min': 172,
  'y_min': 219,
  'z_min': 362,
  'x_max': 184,
  'y_max': 238,
  'z_max': 376},
 {'x_min': 270,
  'y_min': 260,
  'z_min': 267,
  'x_max': 288,
  'y_max': 278,
  'z_max': 283},
 {'x_min': 155,
  'y_min': 175,
  'z_min': 76,
  'x_max': 180,
  'y_max': 204,
  'z_max': 106},
 {'x_min': 279,
  'y_min': 182,
  'z_min': 278,
  'x_max': 303,
  'y_max': 207,
  'z_max': 304},
 {'x_min': 256,
  'y_min': 171,
  'z_min': 135,
  'x_max': 330,
  'y_max': 233,
  'z_max': 194},
 {'x_min': 109,
  'y_min': 71,
  'z_min': 181,
  'x_max': 148,
  'y_max': 106,
  'z_max': 212},
 {'x_min': 209,
  'y_min': 106,
  'z_min': 74,
  'x_max': 251,
  'y_max': 147,
  'z_max': 125},
 {'x_min': 186,
  'y_min': 169,
  'z_min': 93,
  'x_max': 199,
  'y_max': 184,
  'z_max': 104},
 {'x_min': 181,
  'y_min': 196,
  'z_min': 259,
  'x_max': 206,
  'y_max': 223,
  'z_max': 287},
 {'x_min': 215,
  'y_min': 272,
  'z

In [5]:
np.unique(coord_nodule_mask['nodule_mask'], return_counts=True)

(array([0., 1.]), array([50168262,     1650]))

In [6]:
nodule_coordinates = pd.DataFrame(nodule_coordinates)
nodule_coordinates.to_csv('/data/output/msd_nodules.csv')
nodule_coordinates.head()

,x_min,y_min,z_min,x_max,y_max,z_max
0,231,178,78,248,192,95
1,172,219,362,184,238,376
2,270,260,267,288,278,283
3,155,175,76,180,204,106
4,279,182,278,303,207,304


In [7]:
int('aa')

ValueError: invalid literal for int() with base 10: 'aa'

### unique exam

In [ ]:
%%time
img, origin, spacing = load_itk(mask_list[0])

print(np.unique(img, return_counts=True))
print(img.shape)
print(origin, spacing)

In [ ]:
%%time
img2, spacing2 = resample(img, spacing)

print(np.unique(img2, return_counts=True))
print(img2.shape)
print(origin, spacing2)

In [ ]:
%%time
x, y, z = img2.shape

count = 0
coordinates = []
for i in range(x):
    for j in range(y):
        for k in range(z):
             if img2[i][j][k] == 1:
                 coordinates.append((i, j, k))

print(len(coordinates))

In [ ]:
%%time
xs = []
ys = []
zs = []
for a in range(len(coordinates)):
    xs.append(coordinates[a][0])
    ys.append(coordinates[a][1])
    zs.append(coordinates[a][2])

In [ ]:
print(np.min(xs))
print(np.min(ys))
print(np.min(zs))
print(np.max(xs))
print(np.max(ys)) 
print(np.max(zs)) 

In [ ]:
%%time
p = img2.transpose(2, 1, 0)
verts, faces, normals, values = measure.marching_cubes(p, 0)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

mesh = Poly3DCollection(verts[faces], alpha=0.70)
face_color = [1, 0, 0, 0.7]
mesh.set_facecolor(face_color)
ax.add_collection3d(mesh)

### bounding box
axes = [x, y, z]
data = np.zeros(axes)

for i in range(np.min(xs), np.max(xs)):
    for j in range(np.min(ys), np.max(ys)):
        for k in range(np.min(zs), np.max(zs)):
            data[i][j][k] = 1

p = data.transpose(2, 1, 0)
verts, faces, normals, values = measure.marching_cubes(p, 0)

mesh = Poly3DCollection(verts[faces], alpha=0.20)
face_color = [0.45, 0.45, 0.75]
mesh.set_facecolor(face_color)
ax.add_collection3d(mesh)

ax.set_xlim(0, p.shape[0])
ax.set_ylim(0, p.shape[1])
ax.set_zlim(0, p.shape[2])

# ax.view_init(90, 90)

plt.show()